In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from jupyterthemes import jtplot
jtplot.style(theme="monokai", context="notebook", ticks=True, grid=True)

# Load the data

In [5]:
def load_data(sub_sample=True, add_outlier=False):
    """Load data and convert it to the metric system."""
    path_dataset = "height_weight_genders.csv"
    data = np.genfromtxt(path_dataset, delimiter=",", skip_header=1, usecols=[1, 2])
    gender = np.genfromtxt(path_dataset, delimiter=",", skip_header=1, usecols=[0],
                           converters={0:lambda x: 0 if b"Male" in x else 1})
    height = data[:, 0]
    weight = data[:, 1]
    # Convert to metric system
    height *= 0.025
    weight *= 0.454
        # sub-sample
    if sub_sample:
        height = height[::50]
        weight = weight[::50]
    if add_outlier:
        # outlier experiment
        height = np.concatenate([height, [1.1, 1.2]])
        weight = np.concatenate([weight, [51.5/0.454, 55.3/0.454]])
        
    return height, weight, gender

def standardise(x):
    """Standardize the original data set."""
    mean_x = np.mean(x, axis=0)
    x = x - mean_x
    std_x = np.std(x, axis=0)
    x = x / std_x
    return x, mean_x, std_x

def build_model_data(height, weight):
    """Form (y,tX) to get regression data in matrix form."""
    y = weight
    x = height
    num_samples = len(y)
    tx = np.c_[np.ones(num_samples), x]
    return y, tx


In [6]:
import datetime
height, weight, gender = load_data(sub_sample=False, add_outlier=False)
x, mean_x, std_x = standardise(height)
y, tx = build_model_data(x, weight)

In [7]:
y.shape, tx.shape

((10000,), (10000, 2))

# Computing the Cost Function
Fill in the `compute_cost` function below:

In [8]:
def calculate_mse(e):
    """Calculate the mse for vector e."""
    return 1/2*np.mean(e**2)


def calculate_mae(e):
    """Calculate the mae for vector e."""
    return np.mean(np.abs(e))


def compute_loss(y, tx, w):
    """Calculate the loss.

    You can calculate the loss using mse or mae.
    """
    e = y - tx.dot(w)
    return calculate_mse(e)

# Grid Search

Fill in the function `grid_search()` below:

In [9]:
def grid_search(y, tx, w0, w1):
    """Algorithm for grid search."""
    loss = np.zeros((len(w0), len(w1)))
    # compute loss for each combinationof w0 and w1.
    for ind_row, row in enumerate(w0):
        for ind_col, col in enumerate(w1):
            w = np.array([row, col])
            loss[ind_row, ind_col] = compute_loss(y, tx, w)
    return loss

Let us play with the grid search demo now!

In [10]:


def generate_w(num_intervals):
    """Generate a grid of values for w0 and w1."""
    w0 = np.linspace(-100, 200, num_intervals)
    w1 = np.linspace(-150, 150, num_intervals)
    return w0, w1

def get_best_parameters(w0, w1, losses):
    """Get the best w from the result of grid search."""
    min_row, min_col = np.unravel_index(np.argmin(losses), losses.shape)
    return losses[min_row, min_col], w0[min_row], w1[min_col]

def prediction(w0, w1, mean_x, std_x):
    """Get the regression line from the model."""
    x = np.arange(1.2, 2, 0.01)
    x_normalized = (x - mean_x) / std_x
    return x, w0 + w1 * x_normalized

def base_visualisation(grid_losses, w0_list, w1_list,
                       mean_x, std_x, height, weight):
    """Base Visualization for both models."""
    w0, w1 = np.meshgrid(w0_list, w1_list)

    fig = plt.figure()

    # plot contourf
    ax1 = fig.add_subplot(1, 2, 1)
    cp = ax1.contourf(w0, w1, grid_losses.T, cmap=plt.cm.jet)
    fig.colorbar(cp, ax=ax1)
    ax1.set_xlabel(r'$w_0$')
    ax1.set_ylabel(r'$w_1$')
    # put a marker at the minimum
    loss_star, w0_star, w1_star = get_best_parameters(
        w0_list, w1_list, grid_losses)
    ax1.plot(w0_star, w1_star, marker='*', color='r', markersize=20)

    # plot f(x)
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.scatter(height, weight, marker=".", color='b', s=5)
    ax2.set_xlabel("x")
    ax2.set_ylabel("y")
    ax2.grid()

    return fig


def grid_visualisation(grid_losses, w0_list, w1_list,
                       mean_x, std_x, height, weight):
    """Visualize how the trained model looks like under the grid search."""
    fig = base_visualisation(
        grid_losses, w0_list, w1_list, mean_x, std_x, height, weight)

    loss_star, w0_star, w1_star = get_best_parameters(
        w0_list, w1_list, grid_losses)
    # plot prediciton
    x, f = prediction(w0_star, w1_star, mean_x, std_x)
    ax2 = fig.get_axes()[2]
    ax2.plot(x, f, 'r')

    return fig

# Generate the grid of parameters to be swept
grid_w0, grid_w1 = generate_w(num_intervals=250)

# Start the grid search
start_time = datetime.datetime.now()
grid_losses = grid_search(y, tx, grid_w0, grid_w1)


# Gradient Descent (GD)

An element is called a local miminum if $\exists$ $\hat{w}$ s.t. 
$E(\hat{w}) \leq E(w)\,\,\,\,\forall\, w\,\,\,\,with\,\,\, ||w-\hat{w}||\leq\epsilon $.

An element is called a global minimum point if 
$E(\hat{w}) \leq E(w)\,\,\,\,\forall\,w\in \mathbb{R}^n $.
For exmaple, for regression we conclude that
$$\hat{w}=(X^T X)^{-1}X^Ty$$
satisfies
$\hat{w}=arg\,\,\underset{w}{\min } MSE\,(w)$ for $MSE\,(w)=\frac{1}{2s}||Xw-y||^2$

If MSE is convex, then
$\Delta MSE\,(\hat{w})=0$ which results in $MSE\,(\hat{w})\leq MSE\,(w)\,\,\,\,\,\forall\,w\in \mathbb{R}^n$.
However, in case of problems without closed-form solutions , we need to use systematic approaches such as GD.

Smooth functions allow the appliction of more sysytematic search compared to the grid search
$$E\in C^1 (\mathbb{R}^n)\,\,\,\,\,\,\Longrightarrow \Delta E \,\, exists$$
Example of smooth optimisiation is GD:
$$w^{k+1}=w^k-\tau\Delta E(w^k)$$
for some $w_0\in \mathbb{R}^n$ and constant $\tau > 0$.

#### GD for one-parameter MSE-model:    $\,\,\,\,\,\,\text{MSE} w=\frac{1}{2s}{\sum _{j=1}^s \left| w-y_j\right| {}^2}$<br>
Gradient:      $\,\,\,\,\,\Delta MSE(w)-\frac{1}{s} \sum _{j=1}^s y_j $<br>
GD: $\,\,\,\,\,\,w^{k+1}=(1-\tau)w^k+\frac{\tau}{s} \sum _{j=1}^s y_j $

#### For the general linear MSE: $\,\,\,\,\,\,\,\,\,\,\,\,MSE(w)=\frac{1}{2s}\left| Xw-y \right|^2$<br>
Recall: $\,\,\,\Delta MSE(w)=\frac{1}{s}X^T(Xw-y)$<br>
GD: $\,\,\,\,w^{k+1}=w^k+\frac{\tau}{s}X^T(y-Xw^k)=\left(I-\frac{\tau}{s}X^T X \right) y+\frac{\tau}{s}X^T y$

In [11]:
def compute_gradient(y, tx, w):
    """Compute the gradient."""
    err = y - tx.dot(w)
    grad = -tx.T.dot(err) / len(err)
    return grad, err

Please fill in the functions `gradient_descent` below:

In [12]:
def gradient_descent(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # compute loss, gradient
        grad, err = compute_gradient(y, tx, w)
        loss = calculate_mse(err)
        # gradient w by descent update
        w = w - gamma * grad
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws

Test your gradient descent function through gradient descent demo shown below:

In [14]:
# from gradient_descent import *
#from plots import gradient_descent_visualisation

# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.1

# Initialization
w_initial = np.array([0, 0])

# Start gradient descent.
start_time = datetime.datetime.now()
gradient_losses, gradient_ws = gradient_descent(y, tx, w_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

Gradient Descent(0/49): loss=2792.2367127591674, w0=7.3293922002105205, w1=1.347971243498896
Gradient Descent(1/49): loss=2264.635056030003, w0=13.925845180399996, w1=2.5611453626479035
Gradient Descent(2/49): loss=1837.2777140793794, w0=19.862652862570513, w1=3.6530020698820147
Gradient Descent(3/49): loss=1491.118267099375, w0=25.205779776523983, w1=4.63567310639271
Gradient Descent(4/49): loss=1210.7291150455712, w0=30.0145939990821, w1=5.520077039252342
Gradient Descent(5/49): loss=983.6139018819906, w0=34.3425267993844, w1=6.316040578826005
Gradient Descent(6/49): loss=799.6505792194903, w0=38.23766631965648, w1=7.032407764442302
Gradient Descent(7/49): loss=650.6402878628647, w0=41.74329188790136, w1=7.677138231496974
Gradient Descent(8/49): loss=529.9419518639979, w0=44.89835489932174, w1=8.257395651846178
Gradient Descent(9/49): loss=432.176299704916, w0=47.737911609600076, w1=8.779627330160464
Gradient Descent(10/49): loss=352.9861214560597, w0=50.29351264885059, w1=9.24963584

In [16]:
def prediction(w0, w1, mean_x, std_x):
    """Get the regression line from the model."""
    x = np.arange(1.2, 2, 0.01)
    x_normalized = (x - mean_x) / std_x
    return x, w0 + w1 * x_normalized


def base_visualisation(grid_losses, w0_list, w1_list,
                       mean_x, std_x, height, weight):
    """Base Visualization for both models."""
    w0, w1 = np.meshgrid(w0_list, w1_list)

    fig = plt.figure()

    # plot contourf
    ax1 = fig.add_subplot(1, 2, 1)
    cp = ax1.contourf(w0, w1, grid_losses.T, cmap=plt.cm.jet)
    fig.colorbar(cp, ax=ax1)
    ax1.set_xlabel(r'$w_0$')
    ax1.set_ylabel(r'$w_1$')
    # put a marker at the minimum
    loss_star, w0_star, w1_star = get_best_parameters(
        w0_list, w1_list, grid_losses)
    ax1.plot(w0_star, w1_star, marker='*', color='r', markersize=20)

    # plot f(x)
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.scatter(height, weight, marker=".", color='b', s=5)
    ax2.set_xlabel("x")
    ax2.set_ylabel("y")
    ax2.grid()

    return fig


def grid_visualisation(grid_losses, w0_list, w1_list,
                       mean_x, std_x, height, weight):
    """Visualize how the trained model looks like under the grid search."""
    fig = base_visualisation(
        grid_losses, w0_list, w1_list, mean_x, std_x, height, weight)

    loss_star, w0_star, w1_star = get_best_parameters(
        w0_list, w1_list, grid_losses)
    # plot prediciton
    x, f = prediction(w0_star, w1_star, mean_x, std_x)
    ax2 = fig.get_axes()[2]
    ax2.plot(x, f, 'r')

    return fig


def gradient_descent_visualisation(
        gradient_losses, gradient_ws,
        grid_losses, grid_w0, grid_w1,
        mean_x, std_x, height, weight, n_iter=None):
    """Visualize how the loss value changes until n_iter."""
    fig = base_visualisation(
        grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight)

    ws_to_be_plotted = np.stack(gradient_ws)
    if n_iter is not None:
        ws_to_be_plotted = ws_to_be_plotted[:n_iter]

    ax1, ax2 = fig.get_axes()[0], fig.get_axes()[2]
    ax1.plot(
        ws_to_be_plotted[:, 0], ws_to_be_plotted[:, 1],
        marker='o', color='w', markersize=10)
    pred_x, pred_y = prediction(
        ws_to_be_plotted[-1, 0], ws_to_be_plotted[-1, 1],
        mean_x, std_x)
    ax2.plot(pred_x, pred_y, 'r')

    return fig


In [19]:
# Time Visualization
#from plots import *
from ipywidgets import IntSlider, interact
def plot_figure(n_iter):
    fig = gradient_descent_visualisation(
        gradient_losses, gradient_ws, grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight, n_iter)
    fig.set_size_inches(14.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(gradient_ws)))

interactive(children=(IntSlider(value=1, description='n_iter', max=51, min=1), Output()), _dom_classes=('widge…

<function __main__.plot_figure(n_iter)>

# Stochastic gradient descent

In [22]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    err = y - tx.dot(w)
    grad = -tx.T.dot(err) / len(err)
    return grad, err

def stochastic_gradient_descent(
        y, tx, initial_w, batch_size, max_iters, gamma):
    """Stochastic gradient descent."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad, _ = compute_stoch_gradient(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
            loss = compute_loss(y, tx, w)
            # store w and loss
            ws.append(w)
            losses.append(loss)

        print("SGD({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws

In [23]:
# from stochastic_gradient_descent import *

# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.1
batch_size = 15

# Initialization
w_initial = np.array([0, 0])

# Start SGD.
start_time = datetime.datetime.now()
sgd_losses, sgd_ws = stochastic_gradient_descent(
    y, tx, w_initial, batch_size, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SGD: execution time={t:.3f} seconds".format(t=exection_time))

SGD(0/49): loss=2322.0845701589747, w0=6.759649180807213, w1=-0.18000589012225826
SGD(1/49): loss=1851.7338489799918, w0=13.712918388271106, w1=2.398199424997337
SGD(2/49): loss=1479.288724585339, w0=19.85654496630883, w1=4.979570732219344
SGD(3/49): loss=1234.8621301709481, w0=24.89788142792651, w1=3.642246229370392
SGD(4/49): loss=1033.776065517326, w0=29.412720621823546, w1=2.9335984245981757
SGD(5/49): loss=835.2022632975414, w0=33.81867631361184, w1=4.46096924702144
SGD(6/49): loss=649.7802622358078, w0=38.25202188052891, w1=7.0880002540918765
SGD(7/49): loss=548.250092148382, w0=41.41576118307755, w1=6.443287820505029
SGD(8/49): loss=456.9340450184698, w0=44.513557985625276, w1=6.077890944424077
SGD(9/49): loss=382.9168217962344, w0=47.00631185068698, w1=6.844697711428561
SGD(10/49): loss=307.41055565733956, w0=49.703740369814405, w1=8.230649077345982
SGD(11/49): loss=257.56521526770416, w0=51.895971763930774, w1=8.333220386772005
SGD(12/49): loss=207.15055952135194, w0=54.362836

In [24]:
# Time Visualization
from ipywidgets import IntSlider, interact
def plot_figure(n_iter):
    fig = gradient_descent_visualisation(
        sgd_losses, sgd_ws, grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight, n_iter)
    fig.set_size_inches(25.0, 12.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(gradient_ws)))

interactive(children=(IntSlider(value=1, description='n_iter', max=51, min=1), Output()), _dom_classes=('widge…

<function __main__.plot_figure(n_iter)>